In [3]:
import pandas as pd
from dfply import *
import numpy as np


In [4]:
data = pd.read_csv("../crawling/mbti_df_27500-55000.csv")
data.shape

(1069777, 6)

In [5]:
# null 확인
print("na 제거 전 : \n", data.isna().sum())

# 이유 모를 데이터 중복 제거
data.drop_duplicates(inplace = True)

# na 제거
data.dropna(inplace = True)

print(data.shape)
print("na 제거 전 : \n",data.isna().sum())

na 제거 전 : 
 name_list            0
mbti_list            0
gender_list          0
text_list        38359
datetime_list        0
url_list             0
dtype: int64
(1027570, 6)
na 제거 전 : 
 name_list        0
mbti_list        0
gender_list      0
text_list        0
datetime_list    0
url_list         0
dtype: int64


In [6]:
# 이름 정보 없는 데이터 제거
data = data[data['name_list'] != '0']

In [9]:
# mbti, unkn 추출 및 대문자화
data['mbti_list'] = data['mbti_list'].map(lambda x: x.strip()[:4].upper())

# name, 성별 기준 그룹핑 후 text 리스트로 연결
data_gp = pd.DataFrame(data.groupby(['name_list', 'gender_list','mbti_list'])['text_list'].apply(list))

# name별 묶인 list를 |||로 join
data_gp['text_list'] = ["|||".join(text) for text in data_gp['text_list']]

data_gp = data_gp.reset_index()

In [10]:
# mbti 없는 행 제거
data_gp = data_gp[~data_gp['mbti_list'].isin(['','UNKN'])]

# name, gender, mbti 기준 그룹화 했기때문에 이름 기준 중복 가능성 제거
data_gp.drop_duplicates('name_list',inplace = True)

In [11]:
data_gp.columns = ['NAME', 'GENDER', 'MBTI', 'TEXT']

## name 식별화

In [12]:
data_gp['ID'] = range(len(data_gp))

In [13]:
mbti_data = data_gp[['ID', 'GENDER', 'MBTI', 'TEXT']]

In [14]:
mbti_data

ID  GENDER  MBTI                                               TEXT
0          0  Female  INTP  Unfortunately, can't upload your images right ...
1          1    Male  INTJ  Started dating INFP today...I like her...|||IN...
2          2    Male  ENTJ  Combining your anxiety and avoidance scores, y...
3          3    Male  INFP  "I don't understand why you are scared to driv...
4          4  Female  INFJ  I wish I could skip work today. I wish I could...
...      ...     ...   ...                                                ...
29011  25030  Female  INFP  This is what you get when you choose your univ...
29012  25031    Male  ENTP  I hate how arrogant, stubborn and pig headed E...
29013  25032    Male  INTJ  I'm only ever fooled when I don't go with the ...
29014  25033    Male  INFJ  Perplexing.|||.. elusive ..|||Legendary....|||...
29015  25034    Male  ENTP  Being I am a master of one-upping I want to ma...

[25035 rows x 4 columns]